# Environment setup

In [25]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
cd /content/drive/MyDrive/CustomerReviewSentiment

/content/drive/MyDrive/CustomerReviewSentiment


In [27]:
!npm install vietnamese-stopwords

npm WARN saveError ENOENT: no such file or directory, open '/content/drive/MyDrive/CustomerReviewSentiment/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/drive/MyDrive/CustomerReviewSentiment/package.json'
npm WARN CustomerReviewSentiment No description
npm WARN CustomerReviewSentiment No repository field.
npm WARN CustomerReviewSentiment No README data
npm WARN CustomerReviewSentiment No license field.

+ vietnamese-stopwords@0.0.2
updated 1 package and audited 1 package in 0.559s
found 0 vulnerabilities



   ╭────────────────────────────────────────────────────────────────╮
   │                                                                │
   │      New major version of npm available! 6.14.8 → 10.2.5       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v10.2.5   │
   │               Run npm install -g npm to update!                │
   │                                                                │
   ╰─────────────────────────────

In [28]:
!pip install underthesea

In [29]:
!pip install flaml

In [30]:
!pip install sentence_transformers

# Data preparation

In [31]:
import pandas as pd

data = pd.read_csv('final_data_analysis.csv')

In [32]:
data = data.drop([data.columns[0], data.columns[1], data.columns[2], data.columns[3], data.columns[6], data.columns[8]], axis=1)
data

,score,thumbsUpCount,spell_corrected_content
0,1,1,không làm được
1,1,1,đăng xuất ra vô lại bị lỗi nhiều
2,1,0,1 sào vì chăn mà vẫn gỡ được
3,5,1,sạn năm sao luận
4,1,1,mắt ft hoài
...,...,...,...
47842,4,0,ứng dụng tốt theo dõi và làm quen được nhiều b...
47843,4,0,cập nhật không được ạ
47844,4,0,chụp đẹp nhưng chưa có tính năng giấy nhà mong...
47845,1,0,điện video bị lỗi


# Text Preprocessing

In [33]:

from underthesea import word_tokenize
import re


def read_stop_words(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stop_words = [line.strip() for line in file]
    return stop_words

stop_words_path = 'vietnamese-stopwords.txt'
stop_words = set(read_stop_words(stop_words_path))

def clean_text(text):
    if not isinstance(text, str):
        return ''

    text = re.sub(r'[^\w\s]', '', text, re.UNICODE)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if not word in stop_words]

    text = " ".join(tokens)
    return text
data['spell_corrected_content'] = data.spell_corrected_content.apply(lambda x: clean_text(x))


In [34]:
def count_sentences(text):
    sentences = text.split(".")
    return len(sentences)

data['sentence_count'] = data['spell_corrected_content'].apply(count_sentences)
data = data[data['sentence_count'] <= 1].drop(columns=['sentence_count'])

In [35]:
data

,score,thumbsUpCount,spell_corrected_content
0,1,1,
1,1,1,đăng xuất vô lỗi
2,1,0,1 sào chăn gỡ
3,5,1,sạn sao luận
4,1,1,mắt ft hoài
...,...,...,...
47842,4,0,ứng dụng theo dõi làm quen bạn bè
47843,4,0,cập nhật
47844,4,0,chụp đẹp tính năng giấy mong nxb nâng cấp
47845,1,0,điện video lỗi


# Feature Extraction

In [36]:
from sentence_transformers import SentenceTransformer
import pandas as pd
encode_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2', device = 'cuda')
encode_model.eval()
encoded_values = encode_model.encode(list(data['spell_corrected_content']), batch_size = 1000)
encoded_df = pd.DataFrame(encoded_values, columns=[f'feature_{i}' for i in range(encoded_values.shape[1])])
data = pd.concat([data, encoded_df], axis=1)
data

,score,thumbsUpCount,spell_corrected_content,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,1,1,,0.017581,-0.002738,-0.038254,-0.041900,-0.053605,0.027044,-0.002852,...,0.016105,-0.003971,0.019439,0.017352,-0.038702,-0.057055,-0.019953,-0.015060,0.021243,-0.026957
1,1,1,đăng xuất vô lỗi,-0.001382,-0.015146,-0.022524,0.025034,0.008692,-0.008572,-0.013654,...,-0.002070,-0.002258,-0.001869,-0.013223,0.007260,-0.037160,0.007612,-0.002652,-0.017461,-0.022157
2,1,0,1 sào chăn gỡ,0.022646,-0.046243,-0.008854,-0.001807,0.012925,-0.009651,-0.024990,...,0.012161,-0.011289,0.022565,-0.006817,-0.021860,-0.015617,-0.013579,-0.033553,-0.008074,-0.014415
3,5,1,sạn sao luận,-0.002922,-0.003945,-0.058613,-0.024287,-0.012302,0.011380,-0.003966,...,0.004098,-0.002856,-0.011961,0.004939,-0.041821,-0.026152,0.008650,0.019303,-0.006412,-0.040642
4,1,1,mắt ft hoài,0.023850,0.007022,-0.029347,0.015657,0.016633,0.034237,0.003328,...,0.037508,0.018969,0.024057,-0.027593,-0.034753,-0.003954,-0.005604,-0.022591,-0.007400,-0.025065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47842,4,0,ứng dụng theo dõi làm quen bạn bè,-0.012206,0.003083,0.022853,0.000963,0.030868,-0.003793,0.049436,...,-0.026598,0.012320,0.013650,0.028914,0.006928,-0.018601,-0.025527,-0.010396,-0.002518,0.036311
47843,4,0,cập nhật,-0.021586,0.026660,-0.032565,0.021679,-0.033543,0.025007,-0.048813,...,0.018473,-0.008599,0.012079,0.062059,-0.038848,-0.060090,-0.028923,0.019602,0.015731,-0.027878
47844,4,0,chụp đẹp tính năng giấy mong nxb nâng cấp,-0.042553,0.042169,0.014858,0.027910,0.069182,0.019210,0.004559,...,0.021934,-0.011168,0.001019,0.041109,-0.026429,-0.052504,0.042659,0.011267,0.030860,0.014463
47845,1,0,điện video lỗi,0.006360,-0.003856,-0.043570,0.021556,-0.005131,0.014414,0.004588,...,0.033347,0.017082,0.029346,0.018597,0.001188,-0.043680,0.001761,-0.006202,0.026174,0.018210


In [37]:
data

,score,thumbsUpCount,spell_corrected_content,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,1,1,,0.017581,-0.002738,-0.038254,-0.041900,-0.053605,0.027044,-0.002852,...,0.016105,-0.003971,0.019439,0.017352,-0.038702,-0.057055,-0.019953,-0.015060,0.021243,-0.026957
1,1,1,đăng xuất vô lỗi,-0.001382,-0.015146,-0.022524,0.025034,0.008692,-0.008572,-0.013654,...,-0.002070,-0.002258,-0.001869,-0.013223,0.007260,-0.037160,0.007612,-0.002652,-0.017461,-0.022157
2,1,0,1 sào chăn gỡ,0.022646,-0.046243,-0.008854,-0.001807,0.012925,-0.009651,-0.024990,...,0.012161,-0.011289,0.022565,-0.006817,-0.021860,-0.015617,-0.013579,-0.033553,-0.008074,-0.014415
3,5,1,sạn sao luận,-0.002922,-0.003945,-0.058613,-0.024287,-0.012302,0.011380,-0.003966,...,0.004098,-0.002856,-0.011961,0.004939,-0.041821,-0.026152,0.008650,0.019303,-0.006412,-0.040642
4,1,1,mắt ft hoài,0.023850,0.007022,-0.029347,0.015657,0.016633,0.034237,0.003328,...,0.037508,0.018969,0.024057,-0.027593,-0.034753,-0.003954,-0.005604,-0.022591,-0.007400,-0.025065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47842,4,0,ứng dụng theo dõi làm quen bạn bè,-0.012206,0.003083,0.022853,0.000963,0.030868,-0.003793,0.049436,...,-0.026598,0.012320,0.013650,0.028914,0.006928,-0.018601,-0.025527,-0.010396,-0.002518,0.036311
47843,4,0,cập nhật,-0.021586,0.026660,-0.032565,0.021679,-0.033543,0.025007,-0.048813,...,0.018473,-0.008599,0.012079,0.062059,-0.038848,-0.060090,-0.028923,0.019602,0.015731,-0.027878
47844,4,0,chụp đẹp tính năng giấy mong nxb nâng cấp,-0.042553,0.042169,0.014858,0.027910,0.069182,0.019210,0.004559,...,0.021934,-0.011168,0.001019,0.041109,-0.026429,-0.052504,0.042659,0.011267,0.030860,0.014463
47845,1,0,điện video lỗi,0.006360,-0.003856,-0.043570,0.021556,-0.005131,0.014414,0.004588,...,0.033347,0.017082,0.029346,0.018597,0.001188,-0.043680,0.001761,-0.006202,0.026174,0.018210


In [38]:
data= data.drop(columns=['spell_corrected_content'])
data.shape

(47847, 514)

# Splitting the Dataset

In [39]:
!pip install flaml
!pip install ray[tune]==2.4.0
!pip install datasets

In [40]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [41]:
score2label = {1:0,2:0,3:1, 4:2,5:2}
data["class"] = data["score"].apply(lambda x: score2label[x])
data = data.drop(columns = 'score', axis = 0)
dataset = Dataset.from_pandas(data)
trainval_test = dataset.train_test_split(test_size=0.2, seed=0)

trainval_dataset = trainval_test["train"]
test_dataset = trainval_test["test"]
train_val = trainval_dataset.train_test_split(test_size=0.125, seed=0)
train_dataset = train_val["train"]
val_dataset = train_val["test"]

In [42]:
data

,thumbsUpCount,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511,class
0,1,0.017581,-0.002738,-0.038254,-0.041900,-0.053605,0.027044,-0.002852,-0.046511,-0.007231,...,-0.003971,0.019439,0.017352,-0.038702,-0.057055,-0.019953,-0.015060,0.021243,-0.026957,0
1,1,-0.001382,-0.015146,-0.022524,0.025034,0.008692,-0.008572,-0.013654,-0.018333,-0.013906,...,-0.002258,-0.001869,-0.013223,0.007260,-0.037160,0.007612,-0.002652,-0.017461,-0.022157,0
2,0,0.022646,-0.046243,-0.008854,-0.001807,0.012925,-0.009651,-0.024990,-0.019927,0.001555,...,-0.011289,0.022565,-0.006817,-0.021860,-0.015617,-0.013579,-0.033553,-0.008074,-0.014415,0
3,1,-0.002922,-0.003945,-0.058613,-0.024287,-0.012302,0.011380,-0.003966,-0.027524,-0.004654,...,-0.002856,-0.011961,0.004939,-0.041821,-0.026152,0.008650,0.019303,-0.006412,-0.040642,2
4,1,0.023850,0.007022,-0.029347,0.015657,0.016633,0.034237,0.003328,-0.049509,-0.021264,...,0.018969,0.024057,-0.027593,-0.034753,-0.003954,-0.005604,-0.022591,-0.007400,-0.025065,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47842,0,-0.012206,0.003083,0.022853,0.000963,0.030868,-0.003793,0.049436,0.001327,-0.025962,...,0.012320,0.013650,0.028914,0.006928,-0.018601,-0.025527,-0.010396,-0.002518,0.036311,2
47843,0,-0.021586,0.026660,-0.032565,0.021679,-0.033543,0.025007,-0.048813,-0.041854,-0.008337,...,-0.008599,0.012079,0.062059,-0.038848,-0.060090,-0.028923,0.019602,0.015731,-0.027878,2
47844,0,-0.042553,0.042169,0.014858,0.027910,0.069182,0.019210,0.004559,0.000610,-0.017255,...,-0.011168,0.001019,0.041109,-0.026429,-0.052504,0.042659,0.011267,0.030860,0.014463,2
47845,0,0.006360,-0.003856,-0.043570,0.021556,-0.005131,0.014414,0.004588,-0.022795,-0.015173,...,0.017082,0.029346,0.018597,0.001188,-0.043680,0.001761,-0.006202,0.026174,0.018210,0


Dataset({
    features: ['thumbsUpCount', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'f

In [45]:
train_dataset = train_dataset.to_pandas().drop(columns = '__index_level_0__', axis = 0)
lst = [train_dataset]
max_size = train_dataset['class'].value_counts().max()

for class_index, group in train_dataset.groupby('class'):
    lst.append(group.sample(max_size-len(group), replace=True))
train_dataset = pd.concat(lst)
train_dataset = Dataset.from_pandas(train_dataset)

In [47]:
train_dataset

Dataset({
    features: ['thumbsUpCount', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'f

In [48]:
test_dataset

Dataset({
    features: ['thumbsUpCount', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'f

In [49]:
target_col = 'class'

train_df = train_dataset.to_pandas().drop(columns = '__index_level_0__', axis = 0)
val_df = val_dataset.to_pandas().drop(columns = '__index_level_0__', axis = 0)
test_df = test_dataset.to_pandas().drop(columns = '__index_level_0__', axis = 0)

X_train = train_df.drop(columns=[target_col])
y_train = train_df[target_col]

X_val = val_df.drop(columns=[target_col])
y_val = val_df[target_col]

X_test = test_df.drop(columns=[target_col])
y_test = test_df[target_col]


In [50]:
train_df

,thumbsUpCount,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511,class
0,0,-0.017151,-0.004679,-0.028055,0.026930,0.006911,0.013686,-0.035288,-0.014566,0.025809,...,0.012351,0.008509,0.011129,-0.001555,-0.037628,-0.010504,0.005040,0.000230,-0.003275,0
1,0,0.015289,-0.006351,0.014832,0.014614,0.006928,-0.011063,-0.018748,-0.038424,0.003342,...,0.031901,0.023332,0.006876,-0.008106,0.007466,-0.005512,-0.000879,0.002924,-0.014188,2
2,0,0.023541,-0.008837,-0.013200,-0.014032,-0.015306,0.014206,-0.009111,-0.027382,-0.033482,...,-0.034720,-0.016234,0.021567,-0.005491,-0.055883,0.014702,-0.028465,0.028483,-0.011713,2
3,0,-0.028462,-0.022801,-0.056974,0.037219,-0.052863,-0.024137,-0.042682,-0.039631,-0.052941,...,0.027018,0.041338,0.033542,0.006256,-0.035165,-0.028204,0.064896,-0.001106,0.062694,0
4,2,-0.040932,0.005675,-0.050012,0.034869,0.019473,0.035220,-0.013363,-0.027964,-0.055078,...,0.007121,0.047816,0.042011,-0.037659,-0.041696,-0.021197,-0.011019,0.043111,-0.001746,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64189,0,-0.028837,-0.004721,-0.003876,0.053604,0.004541,0.031014,0.001006,0.003621,-0.063446,...,0.026382,0.015611,0.022462,-0.031275,-0.077046,0.000227,0.000380,-0.021059,0.021093,2
64190,0,0.019351,0.000120,-0.031054,-0.032375,-0.039879,0.016433,-0.007361,-0.045977,-0.023210,...,-0.018641,0.016151,0.020549,-0.026941,-0.031238,0.003575,-0.032741,0.004871,-0.014169,2
64191,0,-0.038410,-0.013417,-0.003360,0.019934,0.051477,0.021676,0.006988,0.008903,-0.046980,...,0.018715,0.031123,0.018449,-0.030184,-0.060234,-0.000478,-0.009387,0.004251,0.000867,2
64192,0,0.009364,-0.000353,-0.047033,-0.014968,-0.075325,0.013953,-0.004397,-0.044030,-0.048444,...,-0.006758,0.020424,0.028206,-0.033231,-0.062272,-0.011409,-0.005410,-0.002819,0.005476,2


# Training the XGBoost Classifier

In [58]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
from flaml import AutoML
automl = AutoML()
settings = {
    "time_budget": 3600,
    "metric": 'accuracy',
    "estimator_list": ['xgboost'],
    "task": 'classification',
    "log_file_name": 'experiment.log',
    "seed": 7654321,
}
automl.fit(X_train=X_train, y_train=y_train, **settings)

[flaml.automl.logger: 12-25 17:35:17] {1679} INFO - task = classification
[flaml.automl.logger: 12-25 17:35:17] {1690} INFO - Evaluation method: holdout
[flaml.automl.logger: 12-25 17:35:17] {1788} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 12-25 17:35:17] {1900} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl.logger: 12-25 17:35:17] {2218} INFO - iteration 0, current learner xgboost
[flaml.automl.logger: 12-25 17:35:19] {2344} INFO - Estimated sufficient time budget=100383s. Estimated necessary time budget=100s.
[flaml.automl.logger: 12-25 17:35:19] {2391} INFO -  at 25.2s,	estimator xgboost's best error=0.5055,	best estimator xgboost's best error=0.5055
[flaml.automl.logger: 12-25 17:35:19] {2218} INFO - iteration 1, current learner xgboost
[flaml.automl.logger: 12-25 17:35:21] {2391} INFO -  at 27.2s,	estimator xgboost's best error=0.4996,	best estimator xgboost's best error=0.4996
[flaml.automl.logger: 12-25 17:35:21] {2218} INFO - iteratio

In [59]:
print('Best hyperparmeter config:', automl.best_config)
print('Best loss on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))
print(automl.model.estimator)

Best hyperparmeter config: {'n_estimators': 67, 'max_leaves': 365, 'min_child_weight': 0.001, 'learning_rate': 0.4217353843605563, 'subsample': 0.8909671441156414, 'colsample_bylevel': 0.6967214772271301, 'colsample_bytree': 0.9146480377075876, 'reg_alpha': 0.7497383665245015, 'reg_lambda': 2.5823606759085167}
Best loss on validation data: 0.8645
Training duration of best run: 396.7 s
XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=0.6967214772271301, colsample_bynode=None,
              colsample_bytree=0.9146480377075876, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints=None, learning_rate=0.4217353843605563,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=0, max_leaves=365,
   

# Making Predictions and Evaluating the Model

In [60]:
predictions = automl.predict(X_test)

In [61]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.79      0.83      0.81      6095
           1       0.11      0.09      0.10       645
           2       0.70      0.65      0.68      2830

    accuracy                           0.73      9570
   macro avg       0.54      0.53      0.53      9570
weighted avg       0.72      0.73      0.72      9570



In [62]:
from sklearn.metrics import roc_auc_score
y_test_encoded = le.fit_transform(y_test)
pred_prob = automl.predict_proba(X_test)
print(y_test_encoded.shape)
print(predictions.shape)
auc = roc_auc_score(y_test_encoded, pred_prob, multi_class='ovr')
print('AUC: %.2f' % auc)


(9570,)
(9570,)
AUC: 0.73
